In [1]:
import pandas as pd
from datetime import datetime
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import emoji_def
import Slang
import re
import numpy as np

from torch.utils.data import DataLoader
from itertools import combinations
from torchtext.vocab import GloVe

tokenizer = get_tokenizer('basic_english')


USE_CUDA = torch.cuda.is_available()

if USE_CUDA:
    DEVICE = torch.device('cuda')
    print("Using cuda.")
else:
    DEVICE = torch.device('cpu')
    print("Using cpu.")

seed = 53113    
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if USE_CUDA:
    torch.cuda.manual_seed(seed)

Using cpu.


C:\Users\Daniel\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# from google.colab import drive 
# drive.mount('/content/gdrive')
# PATH = "gdrive/My Drive/nlp21/Project/"

PATH = r'C:\Users\Daniel\Documents\NLP Class\Project\\' 

In [45]:
device = 'cpu'
if device == 'cpu':
    VECTORS_CACHE_DIR = r'C:\Users\Daniel\Documents\NLP Class\project\.vector_cache'
    # Please change above to your cache
else:
    VECTORS_CACHE_DIR = r'C:\Users\Daniel\Documents\NLP Class\Project\.vector_cache'
    # This is the default cache on Colab. Caching may not work
    # as expected on Colab.

glove = GloVe(name='twitter.27B',dim=200)

In [6]:
train_data = pd.read_csv(PATH + "train_dev_same_date_clean.csv")
valid_data = pd.read_csv(PATH + "test_dev_same_date_clean.csv")

train_iter = list(zip(train_data['move'],train_data['body_lower_no_punc_stop_emoj_lem_url_slang'].astype(str)))
valid_iter = list(zip(valid_data['move'],valid_data['body_lower_no_punc_stop_emoj_lem_url_slang'].astype(str)))

train_data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0_x,body,created_utc,id,kind,parent,subreddit,utc_raw,...,volume,move,month,body_lower,body_lower_no_punc,body_lower_no_punc_stop,body_lower_no_punc_stop_emoj,body_lower_no_punc_stop_emoj_lem,body_lower_no_punc_stop_emoj_lem_url,body_lower_no_punc_stop_emoj_lem_url_slang
0,0,145755,166303,He sold for a 10 mil profit already. I like DV...,2021-03-29T19:58:14Z,gsr28j8,t2,t1_gsqpxip,GME,1.617066e+09,...,10075068.0,1,3,he sold for a 10 mil profit already. i like dv...,he sold for a 10 mil profit already i like dvf...,sold 10 mil profit already like dvf dont under...,sold 10 mil profit already like dvf dont under...,sell 10 mil profit already like dvf dont under...,sell 10 mil profit already like dvf dont under...,sell 10 mil profit already like dvf dont under...
1,1,406448,443440,"Honestly, this shit got me up earlier, and goi...",2021-04-01T19:25:47Z,gt3bxxc,t2,t3_mi3eli,GME,1.617323e+09,...,9351417.0,-1,4,"honestly, this shit got me up earlier, and goi...",honestly this shit got me up earlier and going...,honestly shit got earlier going bed earlier pu...,honestly shit got earlier going bed earlier pu...,honestly shit get earlier go bed earlier put u...,honestly shit get earlier go bed earlier put u...,honestly shit get earlier go bed earlier put u...
2,2,226054,673216,I appreciate the confidence. I will try this w...,2021-03-11T19:02:09Z,gqn11oa,t2,t1_gqmz84n,GME,1.615511e+09,...,28402777.0,1,3,i appreciate the confidence. i will try this w...,i appreciate the confidence i will try this wh...,appreciate confidence try next see,appreciate confidence try next see,appreciate confidence try next see,appreciate confidence try next see,appreciate confidence try next see
3,3,319915,1091065,Insider knowledge. Shill idiot doesn't realize...,2021-03-03T17:12:10Z,gplau59,t2,t3_lwuqgq,GME,1.614813e+09,...,19325198.0,1,3,insider knowledge. shill idiot doesn't realize...,insider knowledge shill idiot doesnt realize t...,insider knowledge shill idiot doesnt realize c...,insider knowledge shill idiot doesnt realize c...,insider knowledge shill idiot doesnt realize c...,insider knowledge shill idiot doesnt realize c...,insider knowledge shill idiot doesnt realize c...
4,4,400977,259725,Mods r fuk 🌈🐻,2021-04-05T17:06:11Z,gti44np,t2,t3_mkizw4,GME,1.617660e+09,...,14223605.0,1,4,mods r fuk 🌈🐻,mods r fuk 🌈🐻,mods r fuk 🌈🐻,mods r fuk,mod r fuk,mod r fuk,mod r fuk


In [33]:
train_data["count"] = train_data["body_lower_no_punc_stop_emoj_lem_url_slang"].apply(lambda text: len(tokenizer(str(text))))
valid_data["count"] = valid_data["body_lower_no_punc_stop_emoj_lem_url_slang"].apply(lambda text: len(tokenizer(str(text))))

model_d = max(max(valid_data["count"]),max(train_data["count"]))

train_data["tokens"] = train_data["body_lower_no_punc_stop_emoj_lem_url_slang"].apply(lambda text: tokenizer(str(text)))
valid_data["tokens"] = valid_data["body_lower_no_punc_stop_emoj_lem_url_slang"].apply(lambda text: tokenizer(str(text)))


In [55]:
BATCH_SIZE = 64
def collate_into_embed(batch):

    labels = torch.zeros(len(batch))
    examples = torch.zeros(len(batch),model_d, 200)
    
    for i in range(len(batch)):
        example = batch[i]
        label = example[0]
        if label == -1:
            label = 0
        words = tokenizer(example[1])
        vecs = glove.get_vecs_by_tokens(words)
        
        labels[i] = label
        examples[i,:len(vecs)] =  vecs
    
    return labels, examples

In [61]:
# guidance from https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/3_cnn.ipynb

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        #self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        #embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = text.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [65]:
loss_function = torch.nn.NLLLoss()

def train_an_epoch(dataloader):
    model.train() # Sets the module in training mode.
    log_interval = 500

    for idx, (label, text) in enumerate(dataloader):
        model.zero_grad()
        log_probs = model(text)
        loss = loss_function(log_probs, label.long())
        loss.backward()
        optimizer.step()
        if idx % log_interval == 0 and idx > 0:
            print(f'At iteration {idx} the loss is {loss:.3f}.')
            
def get_accuracy(dataloader):
    hits = 0
    tot = 0
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            examples = batch[1]
            labels = batch[0]
            tot += len(labels)
            estimate = model.forward(examples)
            pred = torch.argmax(estimate, dim=1)
            # there's probably a better way to do this
            output = labels - pred
            for x in range(len(labels)):
                if (labels[x] == 0 and pred[x] == 0) or (labels[x] == 1 and pred[x] == 1):
                    hits +=1
            
    return hits/tot

In [67]:
train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE,
                              shuffle=True, 
                              collate_fn=collate_into_embed)
valid_dataloader = DataLoader(valid_iter, batch_size=BATCH_SIZE,
                              shuffle=False, 
                              collate_fn=collate_into_embed)

#vocab_size = len(vocab)
vocab_size = 0
embedding_dim = 200
n_filters = 100
filter_sizes = [3,5,7]
output_dim = 2
dropout_rate = 0.25
pad_index = 0

model = CNN(vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout_rate, pad_index)



In [68]:
EPOCHS = 100 # epoch
optimizer = optim.Adam(model.parameters(), lr=0.0001)

accuracies=[]
train_accuracies=[]
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train_an_epoch(train_dataloader)
    accuracy = get_accuracy(valid_dataloader)
    train_accuracy = get_accuracy(train_dataloader)
    accuracies.append(accuracy)
    train_accuracies.append(train_accuracy)
    time_taken = time.time() - epoch_start_time
    print()
    print(f'After epoch {epoch} the validation accuracy is {accuracy:.3f}.')
    print(f'After epoch {epoch} the training accuracy is {train_accuracy:.3f}.')
    print()

KeyboardInterrupt: 